In [1]:
import pandas as pd
import os
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, precision_recall_curve, average_precision_score, mean_squared_error, r2_score, mean_absolute_error
from scipy.stats import pearsonr

import numpy as np

# preds

# Load the training set of meta-model
ct_chemberta2_valid2 = pd.read_csv('./chemberta2/results/clintox/chemberta2_valid2_clintox_3_predictions.csv')
ct_molformer_valid2 = pd.read_csv('./molformer/results/clintox/molformer_valid2_clintox_3_epoch26.csv')
ct_molbert_valid2 = pd.read_csv('./molbert/results/cilntox/molbert_valid2_clintox_3.csv')

# Load the test data for each model
ct_chemberta2_test = pd.read_csv('./chemberta2/results/clintox/chemberta2_test_clintox_3_predictions.csv')
ct_molformer_test = pd.read_csv('./molformer/results/clintox/molformer_test_clintox_3_epoch26.csv')
ct_molbert_test = pd.read_csv('./molbert/results/cilntox/molbert_test_clintox_3.csv')

# features

# Load the features from chemberta
ct_chemberta2_features_valid2 = pd.read_csv('./chemberta2/features/clintox/chemberta2_valid2_clintox_3_features.csv')
ct_chemberta2_features_test = pd.read_csv('./chemberta2/features/clintox/chemberta2_test_clintox_3_features.csv')

# Load the features from molformer
ct_molformer_features_valid2 = pd.read_csv('./molformer/features/clintox/molformer_valid2_clintox_3_features.csv')
ct_molformer_features_test = pd.read_csv('./molformer/features/clintox/molformer_test_clintox_3_features.csv')

# Load the features from molbert
ct_molbert_features_valid2 = pd.read_csv('./molbert/features/clintox/molbert_valid2_clintox_3_features.csv')
ct_molbert_features_test = pd.read_csv('./molbert/features/clintox/molbert_test_clintox_3_features.csv')


For Clintox (Classification)

In [2]:
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, average_precision_score

# Preparing the actual and predicted values
# Chemberta2
ct_chemberta_actual = ct_chemberta2_test['CT_TOX']
ct_chemberta_pred = ct_chemberta2_test['y_pred']
ct_chemberta_probs = ct_chemberta2_test[['softmax_class_0_prob', 'softmax_class_1_prob']]

# Molformer
ct_molformer_actual = ct_molformer_test['Actual']
ct_molformer_pred = (ct_molformer_test['Prob_Class_1'] > 0.5).astype(int)
ct_molformer_probs = ct_molformer_test[['Prob_Class_0', 'Prob_Class_1']]

# Molbert
ct_molbert_actual = ct_molbert_test['target']
ct_molbert_pred = ct_molbert_test['pred']
ct_molbert_probs = ct_molbert_test['prob']

# Calculating metrics
ct_metrics_results = {}

for model_name, actual, pred, probs in [("Chemberta2", ct_chemberta_actual, ct_chemberta_pred, ct_chemberta_probs['softmax_class_1_prob']),
                                         ("Molformer", ct_molformer_actual, ct_molformer_pred, ct_molformer_probs['Prob_Class_1']),
                                         ("Molbert", ct_molbert_actual, ct_molbert_pred, ct_molbert_probs)]:
    ct_metrics_results[model_name] = {
        "Accuracy": accuracy_score(actual, pred),
        "F1 Score": f1_score(actual, pred),
        "ROC-AUC": roc_auc_score(actual, probs),
        "PR-AUC": average_precision_score(actual, probs)
    }

ct_metrics_results

{'Chemberta2': {'Accuracy': 0.9731543624161074,
  'F1 Score': 0.8,
  'ROC-AUC': 0.9964028776978417,
  'PR-AUC': 0.9540404040404039},
 'Molformer': {'Accuracy': 0.9328859060402684,
  'F1 Score': 0.375,
  'ROC-AUC': 0.9266187050359712,
  'PR-AUC': 0.5340980864486556},
 'Molbert': {'Accuracy': 0.9172413793103448,
  'F1 Score': 0.14285714285714285,
  'ROC-AUC': 0.9133333333333333,
  'PR-AUC': 0.46665875863902184}}

In [3]:
# Determine the 'smiles' values in chemberta that are not in molbert
missing_smiles = set(ct_chemberta2_valid2['smiles']) - set(ct_molbert_valid2['smiles'])

# Find the indices of these missing 'smiles' in the chemberta DataFrame
missing_indices = ct_chemberta2_valid2.index[ct_chemberta2_valid2['smiles'].isin(missing_smiles)].tolist()

# Drop the rows with these missing indices for chemberta and molformer
ct_chemberta2_valid2 = ct_chemberta2_valid2.drop(missing_indices)
ct_molformer_valid2 = ct_molformer_valid2.drop(missing_indices)
ct_chemberta2_features_valid2 = ct_chemberta2_features_valid2.drop(missing_indices)
ct_molformer_features_valid2 = ct_molformer_features_valid2.drop(missing_indices)
ct_molbert_features_valid2 = ct_molbert_features_valid2.drop(missing_indices)

# do the same for test sets
missing_smiles = set(ct_chemberta2_test['smiles']) - set(ct_molbert_test['smiles'])
missing_indices = ct_chemberta2_test.index[ct_chemberta2_test['smiles'].isin(missing_smiles)].tolist()

ct_chemberta2_test = ct_chemberta2_test.drop(missing_indices)
ct_molformer_test = ct_molformer_test.drop(missing_indices)
ct_chemberta2_features_test = ct_chemberta2_features_test.drop(missing_indices)
ct_molformer_features_test = ct_molformer_features_test.drop(missing_indices)
ct_molbert_features_test = ct_molbert_features_test.drop(missing_indices)

In [4]:
# check shapes
print(ct_chemberta2_valid2.shape)
print(ct_molformer_valid2.shape)
print(ct_molbert_valid2.shape)
print(ct_chemberta2_features_valid2.shape)
print(ct_molformer_features_valid2.shape)
print(ct_molbert_features_valid2.shape)

(295, 8)
(295, 5)
(295, 4)
(295, 386)
(295, 769)
(295, 769)


In [5]:
import pandas as pd
import numpy as np

ct_y_ensemble_valid2 = ct_chemberta2_valid2['CT_TOX']

# Convert the ensemble target to a Series if not already done
ct_y_ensemble_valid2_s = pd.Series(ct_y_ensemble_valid2).reset_index(drop=True)

# Create dataframes for each model's class 1 probability
ct_chemberta2_prob = pd.DataFrame({'chemberta2': ct_chemberta2_valid2['softmax_class_1_prob']})
ct_chemberta2_prob.reset_index(drop=True, inplace=True)

ct_molformer_prob = pd.DataFrame({'molformer': ct_molformer_valid2['Prob_Class_1']})
ct_molformer_prob.reset_index(drop=True, inplace=True)

ct_molbert_prob = pd.DataFrame({'molbert': ct_molbert_valid2['prob']})
ct_molbert_prob.reset_index(drop=True, inplace=True)

# do the same for features ct_chemberta2_features_valid2.iloc[:, 2:]
ct_chemberta2_features = pd.DataFrame(ct_chemberta2_features_valid2.iloc[:, 2:])
ct_chemberta2_features.reset_index(drop=True, inplace=True)

ct_molformer_features = pd.DataFrame(ct_molformer_features_valid2.iloc[:, 1:])
ct_molformer_features.reset_index(drop=True, inplace=True)

ct_molbert_features = pd.DataFrame(ct_molbert_features_valid2.iloc[:, 1:])
ct_molbert_features.reset_index(drop=True, inplace=True)

# ct_features = pd.concat([ct_chemberta2_features, ct_molformer_features, ct_molbert_features], axis=1)

# Combine probabilities into one dataframe
train_ct_prob = pd.concat([ct_chemberta2_prob, ct_molformer_prob, ct_molbert_prob], axis=1)

# Function to calculate BCE for each row
def calculate_bce_rowwise(y_true, y_pred):
    return -(y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred))

# Calculate row-wise BCE for each model
bce_chemberta = calculate_bce_rowwise(ct_y_ensemble_valid2_s, ct_chemberta2_prob['chemberta2'])
bce_molformer = calculate_bce_rowwise(ct_y_ensemble_valid2_s, ct_molformer_prob['molformer'])
bce_molbert = calculate_bce_rowwise(ct_y_ensemble_valid2_s, ct_molbert_prob['molbert'])

# Create a dataframe for row-wise BCE losses
bce_loss_df = pd.DataFrame({
    'bce_chemberta': bce_chemberta,
    'bce_molformer': bce_molformer,
    'bce_molbert': bce_molbert
})

# Final ensemble X matrix: Combine row-wise BCE losses, predictions, and features
ct_X_ensemble_valid2 = pd.concat([bce_loss_df, train_ct_prob], axis=1)

In [6]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import KFold
import numpy as np
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.early_stop import no_progress_loss

# Combine probabilities with their respective feature sets
chemberta_X = pd.concat([ct_chemberta2_prob, ct_chemberta2_features], axis=1)
molformer_X = pd.concat([ct_molformer_prob, ct_molformer_features], axis=1)
molbert_X = pd.concat([ct_molbert_prob, ct_molbert_features], axis=1)

# Standardize each dataset
scaler_chemberta = StandardScaler().fit(chemberta_X)
scaler_molformer = StandardScaler().fit(molformer_X)
scaler_molbert = StandardScaler().fit(molbert_X)

chemberta_X_scaled = scaler_chemberta.transform(chemberta_X)
molformer_X_scaled = scaler_molformer.transform(molformer_X)
molbert_X_scaled = scaler_molbert.transform(molbert_X)

# Define the binary cross-entropy loss values as target variables (y)
chemberta_y_bce = bce_chemberta  # Row-wise BCE loss calculated earlier
molformer_y_bce = bce_molformer  # Row-wise BCE loss calculated earlier
molbert_y_bce = bce_molbert      # Row-wise BCE loss calculated earlier

# Ensure that X_scaled and y_bce are numpy arrays
chemberta_X_scaled = np.array(chemberta_X_scaled)
molformer_X_scaled = np.array(molformer_X_scaled)
molbert_X_scaled = np.array(molbert_X_scaled)

chemberta_y_bce = np.array(chemberta_y_bce)
molformer_y_bce = np.array(molformer_y_bce)
molbert_y_bce = np.array(molbert_y_bce)

torch.manual_seed(0)

# Define custom RMSE loss function
class RMSELoss(nn.Module):
    def __init__(self):
        super(RMSELoss, self).__init__()
        self.mse = nn.MSELoss()

    def forward(self, y_pred, y_true):
        return torch.sqrt(self.mse(y_pred, y_true))

# Define the neural network model
class SimpleNN(nn.Module):
    def __init__(self, input_size, num_layers, num_neurons, dropout_rate):
        super(SimpleNN, self).__init__()
        layers = [nn.Linear(input_size, num_neurons), nn.ReLU(), nn.Dropout(dropout_rate)]
        
        for _ in range(num_layers - 1):
            layers += [nn.Linear(num_neurons, num_neurons), nn.ReLU(), nn.Dropout(dropout_rate)]
        
        layers += [nn.Linear(num_neurons, 1)]  # Output is continuous, no activation function
        
        self.layers = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.layers(x)

# Objective function for Bayesian optimization
def objective(params, X, y):
    kf = KFold(n_splits=5)
    rmses = []

    for train_index, val_index in kf.split(X):
        X_train, X_val = X[train_index], X[val_index]
        y_train, y_val = y[train_index], y[val_index]

        train_dataset = TensorDataset(torch.tensor(X_train.astype(np.float32)), 
                                      torch.tensor(y_train.astype(np.float32)).unsqueeze(1))
        train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

        model = SimpleNN(input_size=X_train.shape[1], num_layers=int(params['num_layers']), 
                         num_neurons=int(params['num_neurons']), dropout_rate=params['dropout_rate'])
        criterion = RMSELoss()  # Use RMSELoss for regression
        optimizer = optim.Adam(model.parameters(), lr=params['learning_rate'])

        model.train()
        for epoch in range(100):  # Set number of epochs
            for inputs, targets in train_loader:
                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs, targets)
                loss.backward()
                optimizer.step()

        model.eval()
        with torch.no_grad():
            X_val_tensor = torch.tensor(X_val.astype(np.float32))
            y_val_tensor = torch.tensor(y_val.astype(np.float32)).unsqueeze(-1)
            outputs = model(X_val_tensor)
            rmse = np.sqrt(mean_squared_error(y_val_tensor.numpy(), outputs.numpy()))
            rmses.append(rmse)

    avg_rmse = np.mean(rmses)
    return {'loss': avg_rmse, 'status': STATUS_OK}  # Minimize RMSE

# Hyperparameter space for optimization
space = {
    'num_layers': hp.quniform('num_layers', 1, 5, 1),
    'num_neurons': hp.quniform('num_neurons', 16, 256, 1),
    'learning_rate': hp.loguniform('learning_rate', np.log(0.0001), np.log(0.01)),
    'dropout_rate': hp.uniform('dropout_rate', 0.0, 0.5)
}

# Scaled input data (after standardization) and corresponding target (binary cross-entropy loss)
datasets = {
    'chemberta': (chemberta_X_scaled, chemberta_y_bce),
    'molformer': (molformer_X_scaled, molformer_y_bce),
    'molbert': (molbert_X_scaled, molbert_y_bce)
}

# Run Bayesian optimization for each model
best_params = {}
trials_dict = {}

for dataset_name, (X_scaled, y_bce) in datasets.items():
    print(f"Optimizing for {dataset_name}...")
    trials = Trials()
    
    best_params[dataset_name] = fmin(fn=lambda params: objective(params, X_scaled, y_bce),
                                     space=space,
                                     algo=tpe.suggest,
                                     max_evals=50,
                                     trials=trials,
                                     rstate=np.random.default_rng(0),  # Seed for reproducibility in hyperopt
                                     early_stop_fn=no_progress_loss(10))
    
    trials_dict[dataset_name] = trials
    print(f"Best hyperparameters for {dataset_name}: {best_params[dataset_name]}")

Optimizing for chemberta...
 32%|███▏      | 16/50 [01:05<02:18,  4.08s/trial, best loss: 0.3201264441013336]
Best hyperparameters for chemberta: {'dropout_rate': 0.3476486742192898, 'learning_rate': 0.004207657605324512, 'num_layers': 3.0, 'num_neurons': 256.0}
Optimizing for molformer...
 40%|████      | 20/50 [01:27<02:11,  4.38s/trial, best loss: 1.6076781749725342]
Best hyperparameters for molformer: {'dropout_rate': 0.12139934126762336, 'learning_rate': 0.006175152235539875, 'num_layers': 3.0, 'num_neurons': 96.0}
Optimizing for molbert...
 32%|███▏      | 16/50 [01:09<02:27,  4.35s/trial, best loss: 0.8250967264175415]
Best hyperparameters for molbert: {'dropout_rate': 0.3476486742192898, 'learning_rate': 0.004207657605324512, 'num_layers': 3.0, 'num_neurons': 256.0}


In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
from sklearn.metrics import mean_squared_error

# Custom RMSE loss function
class RMSELoss(nn.Module):
    def __init__(self):
        super(RMSELoss, self).__init__()
        self.mse = nn.MSELoss()

    def forward(self, y_pred, y_true):
        return torch.sqrt(self.mse(y_pred, y_true))

# Neural network model
class SimpleNN(nn.Module):
    def __init__(self, input_size, num_layers, num_neurons, dropout_rate):
        super(SimpleNN, self).__init__()
        layers = [nn.Linear(input_size, num_neurons), nn.ReLU(), nn.Dropout(dropout_rate)]
        
        for _ in range(num_layers - 1):
            layers += [nn.Linear(num_neurons, num_neurons), nn.ReLU(), nn.Dropout(dropout_rate)]
        
        layers += [nn.Linear(num_neurons, 1)]  # Output layer
        
        self.layers = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.layers(x)

# Function to train the model using the best hyperparameters
def train_model(X_train, y_train, best_params):
    # Create DataLoader for training data
    train_dataset = TensorDataset(torch.tensor(X_train.astype(np.float32)), 
                                  torch.tensor(y_train.astype(np.float32)).unsqueeze(1))
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

    # Initialize the model with the best hyperparameters
    model = SimpleNN(input_size=X_train.shape[1],
                     num_layers=int(best_params['num_layers']),
                     num_neurons=int(best_params['num_neurons']),
                     dropout_rate=best_params['dropout_rate'])
    
    # Define the loss and optimizer
    criterion = RMSELoss()
    optimizer = optim.Adam(model.parameters(), lr=best_params['learning_rate'])

    # Train the model
    model.train()
    for epoch in range(100):  # Training for 100 epochs
        for inputs, targets in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()

    return model

# Load your best hyperparameters from the optimization step
# Replace these with the actual best hyperparameters you found
best_params_chemberta = best_params['chemberta']
best_params_molformer = best_params['molformer']
best_params_molbert = best_params['molbert']

# Train the models for each dataset
best_model_chemberta = train_model(chemberta_X_scaled, chemberta_y_bce, best_params_chemberta)
best_model_molformer = train_model(molformer_X_scaled, molformer_y_bce, best_params_molformer)
best_model_molbert = train_model(molbert_X_scaled, molbert_y_bce, best_params_molbert)

In [8]:
import numpy as np
from sklearn.metrics import log_loss

# Test data for each model
ct_chemberta2_prob_test = pd.DataFrame({'chemberta2': ct_chemberta2_test['softmax_class_1_prob']})
ct_chemberta2_prob_test.reset_index(drop=True, inplace=True)

ct_molformer_prob_test = pd.DataFrame({'molformer': ct_molformer_test['Prob_Class_1']})
ct_molformer_prob_test.reset_index(drop=True, inplace=True)

ct_molbert_prob_test = pd.DataFrame({'molbert': ct_molbert_test['prob']})
ct_molbert_prob_test.reset_index(drop=True, inplace=True)

ct_chemberta2_features_t = pd.DataFrame(ct_chemberta2_features_test.iloc[:, 2:])
ct_chemberta2_features_t.reset_index(drop=True, inplace=True)

ct_molformer_features_t  = pd.DataFrame(ct_molformer_features_test.iloc[:, 1:])
ct_molformer_features_t.reset_index(drop=True, inplace=True)

ct_molbert_features_t = pd.DataFrame(ct_molbert_features_test.iloc[:, 1:])
ct_molbert_features_t.reset_index(drop=True, inplace=True)

# Combine probabilities with the respective feature sets for the test set
chemberta_X_test = pd.concat([ct_chemberta2_prob_test, ct_chemberta2_features_t], axis=1)
molformer_X_test = pd.concat([ct_molformer_prob_test, ct_molformer_features_t], axis=1)
molbert_X_test = pd.concat([ct_molbert_prob_test, ct_molbert_features_t], axis=1)

# Standardize the test set based on the previously fitted scalers
chemberta_X_test_scaled = scaler_chemberta.transform(chemberta_X_test)
molformer_X_test_scaled = scaler_molformer.transform(molformer_X_test)
molbert_X_test_scaled = scaler_molbert.transform(molbert_X_test)

# Ensure that X_scaled and y_bce are numpy arrays
chemberta_X_test_scaled= np.array(chemberta_X_test_scaled)
molformer_X_test_scaled = np.array(molformer_X_test_scaled)
molbert_X_test_scaled = np.array(molbert_X_test_scaled)

# Predict on the test sets (assuming your test sets are already preprocessed and scaled)
with torch.no_grad():
    chemberta_X_test_tensor = torch.tensor(chemberta_X_test_scaled.astype(np.float32))
    molformer_X_test_tensor = torch.tensor(molformer_X_test_scaled.astype(np.float32))
    molbert_X_test_tensor = torch.tensor(molbert_X_test_scaled.astype(np.float32))
    
    chemberta_pred_test = best_model_chemberta(chemberta_X_test_tensor).numpy()
    molformer_pred_test = best_model_molformer(molformer_X_test_tensor).numpy()
    molbert_pred_test = best_model_molbert(molbert_X_test_tensor).numpy()
        
    # Ensure the predictions are 1-dimensional by flattening if necessary
    chemberta_pred_test = chemberta_pred_test.flatten()
    molformer_pred_test = molformer_pred_test.flatten()
    molbert_pred_test = molbert_pred_test.flatten()
    
# The predictions are now stored in chemberta_pred_test, molformer_pred_test, molbert_pred_test

# Convert the predictions (numpy arrays) to pandas Series
chemberta_pred_test_series = pd.Series(chemberta_pred_test, name='bce_chemberta')
molformer_pred_test_series = pd.Series(molformer_pred_test, name='bce_molformer')
molbert_pred_test_series = pd.Series(molbert_pred_test, name='bce_molbert')

# Now concatenate the series with the test set probabilities
ct_X_ensemble_test = pd.concat([
    chemberta_pred_test_series,                     # BCE for Chemberta
    molformer_pred_test_series,                     # BCE for Molformer
    molbert_pred_test_series,                       # BCE for Molbert
    ct_chemberta2_prob_test['chemberta2'],        # Chemberta test probabilities
    ct_molformer_prob_test['molformer'],          # Molformer test probabilities
    ct_molbert_prob_test['molbert']               # Molbert test probabilities
], axis=1)

ct_X_ensemble_test.columns = ['bce_chemberta', 'bce_molformer', 'bce_molbert', 'chemberta2', 'molformer', 'molbert']

# optional for evaluation
ct_y_ensemble_test = ct_chemberta2_test['CT_TOX']

In [9]:
# use standard scaler
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
ct_X_ensemble_valid2_scaled = scaler.fit_transform(ct_X_ensemble_valid2)
ct_X_ensemble_test_scaled = scaler.transform(ct_X_ensemble_test)

# transform back to dataframe
ct_X_ensemble_valid2_scaled = pd.DataFrame(ct_X_ensemble_valid2_scaled, columns=ct_X_ensemble_valid2.columns)
ct_X_ensemble_test_scaled = pd.DataFrame(ct_X_ensemble_test_scaled, columns=ct_X_ensemble_test.columns)


In [10]:
ct_X_ensemble_valid2_selected = ct_X_ensemble_valid2_scaled
ct_X_ensemble_test_selected = ct_X_ensemble_test_scaled
# check shapes
print(ct_X_ensemble_valid2_selected.shape)
print(ct_X_ensemble_test_selected.shape)

(295, 6)
(145, 6)


In [11]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import KFold
import numpy as np
from sklearn.metrics import roc_auc_score
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.early_stop import no_progress_loss

torch.manual_seed(0)

# Define the neural network model
class SimpleNN(nn.Module):
    def __init__(self, input_size, num_layers, num_neurons, dropout_rate):
        super(SimpleNN, self).__init__()
        layers = [nn.Linear(input_size, num_neurons), nn.ReLU(), nn.Dropout(dropout_rate)]
        
        for _ in range(num_layers - 1):
            layers += [nn.Linear(num_neurons, num_neurons), nn.ReLU(), nn.Dropout(dropout_rate)]
        
        layers += [nn.Linear(num_neurons, 1), nn.Sigmoid()]
        
        self.layers = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.layers(x)

# Objective function for Bayesian optimization
def objective(params):
    kf = KFold(n_splits=5)
    roc_aucs = []

    for train_index, val_index in kf.split(X):
        X_train, X_val = X.iloc[train_index], X.iloc[val_index]
        y_train, y_val = y.iloc[train_index], y.iloc[val_index]

        train_dataset = TensorDataset(torch.tensor(X_train.values.astype(np.float32)), 
                                      torch.tensor(y_train.values.astype(np.float32)).unsqueeze(1))
        train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

        model = SimpleNN(input_size=X_train.shape[1], num_layers=int(params['num_layers']), 
                         num_neurons=int(params['num_neurons']), dropout_rate=params['dropout_rate'])
        criterion = nn.BCELoss()
        optimizer = optim.Adam(model.parameters(), lr=params['learning_rate'])

        model.train()
        for epoch in range(100):
            for inputs, targets in train_loader:
                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs, targets)
                loss.backward()
                optimizer.step()

        model.eval()
        with torch.no_grad():
            X_val_tensor = torch.tensor(X_val.values.astype(np.float32))
            y_val_tensor = torch.tensor(y_val.values.astype(np.float32)).unsqueeze(-1)
            outputs = model(X_val_tensor)
            roc_auc = roc_auc_score(y_val_tensor.numpy(), outputs.numpy())
            roc_aucs.append(roc_auc)

    avg_roc_auc = np.mean(roc_aucs)
    return {'loss': -avg_roc_auc, 'status': STATUS_OK}  # Maximize ROC AUC by minimizing the negative ROC AUC

# Hyperparameter space
space = {
    'num_layers': hp.quniform('num_layers', 1, 5, 1),
    'num_neurons': hp.quniform('num_neurons', 16, 256, 1),
    'learning_rate': hp.loguniform('learning_rate', np.log(0.0001), np.log(0.01)),
    'dropout_rate': hp.uniform('dropout_rate', 0.0, 0.5)
}

X = ct_X_ensemble_valid2_selected
y = ct_y_ensemble_valid2

# Run Bayesian optimization
trials = Trials()
ct_nn_best_params = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=50,
            trials=trials,
            rstate=np.random.default_rng(0),  # Seed for reproducibility in hyperopt
            early_stop_fn=no_progress_loss(10))

print("Best hyperparameters:", ct_nn_best_params)


 20%|██        | 10/50 [00:27<01:51,  2.78s/trial, best loss: -1.0]
Best hyperparameters: {'dropout_rate': 0.4175299871186762, 'learning_rate': 0.00024043206714908388, 'num_layers': 2.0, 'num_neurons': 180.0}


In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
from sklearn.metrics import roc_auc_score

torch.manual_seed(0)

# Define the neural network model again
class SimpleNN(nn.Module):
    def __init__(self, input_size, num_layers, num_neurons, dropout_rate):
        super(SimpleNN, self).__init__()
        layers = [nn.Linear(input_size, num_neurons), nn.ReLU(), nn.Dropout(dropout_rate)]
        
        for _ in range(num_layers - 1):
            layers += [nn.Linear(num_neurons, num_neurons), nn.ReLU(), nn.Dropout(dropout_rate)]
        
        layers += [nn.Linear(num_neurons, 1), nn.Sigmoid()]
        
        self.layers = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.layers(x)

# Convert parameters to the correct format if necessary
ct_nn_best_params = {
    'num_layers': int(ct_nn_best_params['num_layers']),  # Extracted from Bayesian optimization results
    'num_neurons': int(ct_nn_best_params['num_neurons']),  # Extracted from Bayesian optimization results
    'dropout_rate': ct_nn_best_params['dropout_rate'],  # Extracted from Bayesian optimization results
    'learning_rate': ct_nn_best_params['learning_rate']  # Extracted from Bayesian optimization results
}

# Prepare datasets
X_train_tensor = torch.tensor(ct_X_ensemble_valid2_selected.values.astype(np.float32))
y_train_tensor = torch.tensor(ct_y_ensemble_valid2.values.astype(np.float32)).unsqueeze(1)
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

X_test_tensor = torch.tensor(ct_X_ensemble_test_selected.values.astype(np.float32))
y_test_tensor = torch.tensor(ct_y_ensemble_test.values.astype(np.float32)).unsqueeze(1)

# Initialize the model
model = SimpleNN(input_size=ct_X_ensemble_valid2_selected.shape[1], num_layers=ct_nn_best_params['num_layers'], 
                 num_neurons=ct_nn_best_params['num_neurons'], dropout_rate=ct_nn_best_params['dropout_rate'])

criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=ct_nn_best_params['learning_rate'])

# Training loop
model.train()
for epoch in range(100):  # Number of epochs can be adjusted
    for inputs, targets in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

# Evaluation on test set
model.eval()
with torch.no_grad():
    outputs = model(X_test_tensor)
    predictions = (outputs > 0.5).float()

    # Calculate metrics
    accuracy = accuracy_score(y_test_tensor.numpy(), predictions.numpy())
    f1 = f1_score(y_test_tensor.numpy(), predictions.numpy())
    roc_auc = roc_auc_score(y_test_tensor.numpy(), outputs.numpy())
    pr_auc = average_precision_score(y_test_tensor.numpy(), outputs.numpy())

    ct_nn_metrics = {
        'Accuracy': accuracy,
        'F1 Score': f1,
        'ROC-AUC': roc_auc,
        'PR-AUC': pr_auc
    }

ct_nn_metrics

{'Accuracy': 0.9586206896551724,
 'F1 Score': 0.5714285714285714,
 'ROC-AUC': 0.9992592592592593,
 'PR-AUC': 0.9909090909090909}

In [13]:
# report all the metrics for ct
ct_metrics_results["Neural Network"] = ct_nn_metrics

ct_metrics_df = pd.DataFrame(ct_metrics_results).T

# keep 3 digits after the decimal point
ct_metrics_df = ct_metrics_df.round(3)

# export as csv
ct_metrics_df.to_csv('./split3_ct_metrics_nn.csv')